In [ ]:
## This notebook is used to average the monthly ERA5 sea level pressure data to get yearly data
## Created by Kendra Herweck

In [1]:
import netCDF4 as nc
import numpy as np
import datetime as dt  # Python standard library datetime  module
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from astropy.table import Table
from astropy.io import ascii
from pandas import read_csv
from matplotlib.dates import DateFormatter
from pandas import DataFrame
from pandas import Grouper
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import xarray as xr
from glob import glob

## Avg monthly to get yearly

In [2]:
filename1 = 'Data/Sea-Level-Pressure/ERA/adaptor.mars.internal-1625493258.5067286-27348-1-dbbf1d20-ef07-436a-b22b-470dabd4871b_1950-1978.nc'
ds1 = xr.open_dataset(filename1)
filename2 = 'Data/Sea-Level-Pressure/ERA/adaptor.mars.internal-1625493210.7690952-661-4-9904a93a-aa6b-4c23-a1b0-bda46b065056_1979-2020.nc'
ds2 = xr.open_dataset(filename2)
ds = xr.merge([ds1,ds2])

In [7]:
print(ds['msl'])

<xarray.DataArray 'msl' (time: 286, latitude: 321, longitude: 1440)>
array([[[101456.41 , 101456.41 , 101456.41 , ..., 101456.41 ,
         101456.41 , 101456.41 ],
        [101473.64 , 101473.64 , 101473.64 , ..., 101473.87 ,
         101473.64 , 101473.64 ],
        [101488.09 , 101488.09 , 101488.09 , ..., 101488.2  ,
         101488.2  , 101488.2  ],
        ...,
        [100968.35 , 100966.91 , 100966.79 , ..., 100969.57 ,
         100968.91 , 100968.79 ],
        [100959.68 , 100959.45 , 100959.01 , ..., 100960.68 ,
         100960.56 , 100960.234],
        [100952.23 , 100952.336, 100951.89 , ..., 100952.12 ,
         100952.336, 100952.23 ]],

       [[101209.266, 101209.266, 101209.266, ..., 101209.266,
         101209.266, 101209.266],
        [101225.945, 101225.945, 101226.055, ..., 101225.72 ,
         101225.83 , 101225.83 ],
        [101244.29 , 101244.4  , 101244.51 , ..., 101244.06 ,
         101244.06 , 101244.17 ],
...
        [101038.42 , 101038.99 , 101040.36 , ...

In [8]:
ds_test = ds.groupby('time.year').mean('time',skipna=True)
print(ds_test)

<xarray.Dataset>
Dimensions:    (latitude: 321, longitude: 1440, year: 72)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... 10.5 10.25 10.0
  * year       (year) int64 1950 1951 1952 1953 1954 ... 2018 2019 2020 2021
Data variables:
    msl        (year, latitude, longitude) float32 1.017e+05 ... 1.01e+05


In [12]:
for i in range(0,len(ds_test['year'])): ##len(ds_test['year'])
    year = ds_test['year'][i].values
    filename = 'Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-{}.nc'.format(year)
    print(filename)
    f = nc.Dataset(filename, 'w')
    f.createDimension('time', 1)
    f.createDimension('y', len(ds_test['latitude']))
    f.createDimension('x', len(ds_test['longitude']))
    lats = f.createVariable('lat', float, ('y', ), zlib=True)
    #print(lats.shape)
    lons = f.createVariable('lon', float, ('x', ), zlib=True)
    slp = f.createVariable('slp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)
    time = f.createVariable('time',float,('time'),zlib=True)
    
    lat_out  = np.array(ds_test.variables['latitude'])
    lon_out  = ds_test.variables['longitude']

    lats[:] = lat_out
    lons[:] = lon_out
    slp[:] = ds_test['msl'][i,:,:].values
    time[:] = ds_test['year'][i].values

    f.close()

Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1950.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1951.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1952.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1953.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1954.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1955.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1956.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1957.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1958.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1959.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1960.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1961.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1962.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1963.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1964.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1965.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1966.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1967.nc
Data/Sea-Level-Pressure/ERA/Yearly/ERA-SLP-1968.nc
Data/Sea-Level-Pressure/ERA/Yea

In [10]:
f.close()